# Реализация первой версии агента

In [1]:
!pip install -q langchain_core
!pip install -q langchain_community
!pip install -q langchain_openai
!pip install -q langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 

In [2]:
!pip install -q tavily

ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily


In [3]:
API_KEY = "апи ключ в Google AI Studio"
MODEL = "gemini-2.0-flash" # в ситуациях, когда заканчивались бесплатные запросы в день, менял на gemini-2.5-flash

In [4]:
from langchain_openai import ChatOpenAI 
from langgraph.graph import StateGraph, END, MessagesState 
from langgraph.prebuilt import ToolNode 
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_community.tools import TavilySearchResults
import json

TAVILY_API_KEY = "ваш ключ"
search_tool = TavilySearchResults(tavily_api_key=TAVILY_API_KEY,
                                  max_results=3, 
                                  include_answer=True, 
                                  include_raw_content=False)
tools = [search_tool] # только ходит в поиск
llm = ChatOpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=API_KEY,
    model=MODEL,
    temperature=0
).bind_tools(tools)

/tmp/ipykernel_35/163455660.py:9: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(tavily_api_key=TAVILY_API_KEY,


In [5]:
# некоторые дальнейшие функции могут быть реализованы костыльно, так как предназначены для предотвращения частых ошибок обработки сообщений от Tavily, зацикливания модели в поиске и слишком большого и несвязного для модели контекста
def tool_message_to_system_message(tool_msg: ToolMessage): # извлекаем из ToolMessage Tavily резы поиска
    try:
        results = json.loads(tool_msg.content)
        content = "### Результаты поисковых запросов:\n\n"
        for j, result in enumerate(results[:3]):
            content += f"- Заголовок: {result.get('title', 'Нет данных')}\n"
            result_content = result.get('content', 'Пусто')
            if len(result_content) > 500:
                result_content = result_content[:500] + "..."
            content += f"- Контент: {result_content}\n\n"  
        return content
    except Exception as e:
        return ""

def collect_results(messages):
    indices_to_remove = []
    results=''
    queries=''
    global prev_res
    global prev_queries
    global queries_count
    for i, msg in enumerate(messages):
        if isinstance(msg, ToolMessage):
            res = tool_message_to_system_message(msg)
            results+='\n'+res # собираем результаты всех предыдущих запросов
            indices_to_remove.append(i) # для предотвращения ошибок обработки ToolMessage Tavily
        elif isinstance(msg, AIMessage) and msg.tool_calls:
            for tool_call in msg.tool_calls:
                query = tool_call['args'].get('query', '')
                queries+='\n'+query # собираем все предыдущие запросы (чтобы модель знала, что спрашивала ранее)
                queries_count+=1 # число запросов нужно для контроля
            indices_to_remove.append(i) # удаляем все вызовы инструментов (будем всю эту информацию агрегировать в промпте)
        elif isinstance(msg, AIMessage) or isinstance(msg, SystemMessage): # рассуждения без вызова инструмента и предыдущие промпты удаляем, чтобы не нагромождать контекст модели
            indices_to_remove.append(i)
    for index in sorted(indices_to_remove, reverse=True):
        del messages[index]
    prev_res+=results
    prev_queries+=queries
    return prev_res, prev_queries

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return END

def gather_data(state: MessagesState):
    global queries_count
    messages = state["messages"]
    results, queries = collect_results(messages)
    if queries_count<2: # разрешаем агенту ходить в поиск не более 2 раз
        messages.append(SystemMessage(content=f'''Ты - аналитик сервиса Яндекс.Карты
Твоя задача: оценить релевантность запроса пользователя найденному месту на карте
Тебе будет предоставлен сам запрос пользователя и данные конкретного места:
*Адрес
*Название
*Нормализованное название
*Цены
*Отзывы
Для поиска данных о месте используй инструмент TavilySearchResults. При использовании поиска
Если ты ранее использовал TavilySearchResults, то вот результаты поиска:
{results}

Ранее ты сделал следующие запросы:
{queries}

Всего ты сделал запросов: {queries_count}

Правила поиска с помощью TavilySearchResults:
1) Найди всю необходимую информацию о месте и запросе пользователя для оценки релевантности места запросу
2) Если ты ранее использовал TavilySearchResults, но информации недостаточно, то продолжай поиск
3) Если после нескольких однотипных запросов информация не находится, то прекрати попытки ее искать
4) Нельзя совершать более 2 запросов: если число твоих запросов ({queries_count}) больше или равно 2, то не делай больше запросов
5) Нельзя предоставлять финальный ответ без, как минимум, одного запроса
    
После того, как вся информация будет собрана, тебе нужно шаг за шагом проанализировать запрос пользователя и найденное место
Особый акцент сделай на пунктах:
1) Тип места, которое ищет пользователь: он должен ПОЛНОСТЬЮ совпадать с искомым, даже если по описанию оно может подходить пользователю. Пример: **ресторан** в Москве -> **ресторан** "Магадан" в Москве (релевантно), **ресторан** в Москве -> дорогой **бар** с подачей ресторанных блюд в Москве (нерелевантно, хотя в найденном баре подают еду из ресторана, он не подходит, потому что бар и ресторан - разные вещи)
2) Слишком обобщенный запрос: нужно исключить ситуации частичной релевантности, общий запрос -> место с общими возможностями. Пользователь мог иметь ввиду что-то другое, тогда узкоспециализированное место ему не подойдет -> такое место нерелевантно общему запросу. Пример: **магазин** -> **супермаркет** (релевантно, так общее место соответствует общему запросу), **магазин** -> магазин **игрушек** (нерелевантно, так как пользователь мог искать магазин с другим ассортиментом)
3) Географическое расположение. Пример: школа в **Москве** -> Школа №1252 имени Сервантеса в **Москве** (релевантно), школа в **Москве** -> школа в **Подольске** (нерелевантно)
4) Доступны ВСЕ запрашиваемые возможности: если хотя бы какая-то особенность отсутствует, то нерелевантно. Пример: парк с **качелями** и **лавочками** -> парк, в котором есть и **качели**, и **лавочки** (релевантно), парк с **качелями** и **лавочками** -> парк, в котором есть **только качели** (нерелевантно)
5) Ты точно понимаешь суть запроса и места и все понятия в запросе: если есть какие-либо малейшие сомнения, то лучше уточни в интернете. Пример: антикафе -> кафе, где платят не за еду, а за время (релевантно, место по определению соответствует запросу), море рядом с Россией -> Каспийское море (нерелевантно, Каспийское море - озеро, а не море)
6) Скептично относись к частичной релевантности: частичная релевантность с вероятностью 90% сведетельствует о НЕРЕЛЕВАНТНОСТИ. Нельзя додумывать за пользователя, что он хотел, если в запросе эта информация отсутствует: так ты будешь пытаться привести частичную релевантность к полной релевантности - это приведет к ошибке
Чтобы учесть все нюансы, правильно пользуйся поиском
В качестве ответа выпиши всю цепочку своих рассуждений и доводов, а затем: answer(1), если найденное место релевантно запросу, answer(0) в противном случае
Доводы должны содержать ссылки на найденную информацию с помощью TavilySearchResults и на данные найденного места. Также нужно пройтись по всем пунктам 1-6 и доказать, что место релевантно или нет относительно КАЖДОГО пункта, и нужно доказать, что ты соблюдал все 5 правил поиска с помощью TavilySearchResults
Никакой другой информации в ответе быть не должно, только цепочка рассуждений и answer(1) или answer(0)
        '''))
    if queries_count>=2: # будем явно запрещать агенту превышать лимит запросов, потому что правила использования поиска он иногда игнорирует 
        messages.append(SystemMessage(content=f'''Ты - аналитик сервиса Яндекс.Карты
Твоя задача: оценить релевантность запроса пользователя найденному месту на карте
Тебе будет предоставлен сам запрос пользователя и данные конкретного места:
*Адрес
*Название
*Нормализованное название
*Цены
*Отзывы
Ты ранее использовал TavilySearchResults, вот результаты поиска:
{results}

Ранее ты сделал следующие запросы:
{queries}
    
Тебе нужно шаг за шагом проанализировать запрос пользователя, найденное место и полученную с помощью TavilySearchResults дополнительную информацию
Особый акцент сделай на пунктах:
1) Тип места, которое ищет пользователь: он должен ПОЛНОСТЬЮ совпадать с искомым, даже если по описанию оно может подходить пользователю. Пример: **ресторан** в Москве -> **ресторан** "Магадан" в Москве (релевантно), **ресторан** в Москве -> дорогой **бар** с подачей ресторанных блюд в Москве (нерелевантно, хотя в найденном баре подают еду из ресторана, он не подходит, потому что бар и ресторан - разные вещи)
2) Слишком обобщенный запрос: нужно исключить ситуации частичной релевантности, общий запрос -> место с общими возможностями. Пользователь мог иметь ввиду что-то другое, тогда узкоспециализированное место ему не подойдет -> такое место нерелевантно общему запросу. Пример: **магазин** -> **супермаркет** (релевантно, так общее место соответствует общему запросу), **магазин** -> магазин **игрушек** (нерелевантно, так как пользователь мог искать магазин с другим ассортиментом)
3) Географическое расположение. Пример: школа в **Москве** -> Школа №1252 имени Сервантеса в **Москве** (релевантно), школа в **Москве** -> школа в **Подольске** (нерелевантно)
4) Доступны ВСЕ запрашиваемые возможности: если хотя бы какая-то особенность отсутствует, то нерелевантно. Пример: парк с **качелями** и **лавочками** -> парк, в котором есть и **качели**, и **лавочки** (релевантно), парк с **качелями** и **лавочками** -> парк, в котором есть **только качели** (нерелевантно)
5) Ты точно понимаешь суть запроса и места и все понятия в запросе: если есть какие-либо малейшие сомнения, то лучше уточни в интернете. Пример: антикафе -> кафе, где платят не за еду, а за время (релевантно, место по определению соответствует запросу), море рядом с Россией -> Каспийское море (нерелевантно, Каспийское море - озеро, а не море)
6) Скептично относись к частичной релевантности: частичная релевантность с вероятностью 90% сведетельствует о НЕРЕЛЕВАНТНОСТИ. Нельзя додумывать за пользователя, что он хотел, если в запросе эта информация отсутствует: так ты будешь пытаться привести частичную релевантность к полной релевантности - это приведет к ошибке
Чтобы учесть все нюансы, правильно пользуйся поиском
В качестве ответа выпиши всю цепочку своих рассуждений и доводов, а затем: answer(1), если найденное место релевантно запросу, answer(0) в противном случае
Доводы должны содержать ссылки на найденную информацию с помощью TavilySearchResults и на данные найденного места. Также нужно пройтись по всем пунктам 1-6 и доказать, что место релевантно или нет относительно КАЖДОГО пункта
Никакой другой информации в ответе быть не должно, только цепочка рассуждений и answer(1) или answer(0)
Тебе ЗАПРЕЩЕНО совершать дальнейшие запросы в Tavily!
        '''))
    response = llm.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)
workflow = StateGraph(MessagesState)
workflow.add_node("gather_data_node", gather_data)
workflow.add_node("tools", tool_node)
workflow.set_entry_point("gather_data_node")
workflow.add_conditional_edges("gather_data_node", 
                               should_continue, 
                               {"tools": "tools", END: END})
workflow.add_edge("tools", "gather_data_node")
graph = workflow.compile()

In [6]:
import re
import numpy as np
def extract_answer_number(output):
    match = re.search(r"answer\((\d+)\)", output['messages'][-1].content)
    if match:
        return int(match.group(1))
    else:
        return np.NaN

In [8]:
import pandas as pd
data = pd.read_csv('/kaggle/input/dls-project-ds/eval_maps.csv')
data

,Text,full,relevance
0,сигары,"[Address]\n Москва, Дубравная улица, 34/29\n ...",1.0
1,кальянная спб мероприятия,"[Address]\n Санкт-Петербург, Большой проспект...",0.0
2,Эпиляция,"[Address]\n Московская область, Одинцово, ули...",1.0
3,стиральных машин,"[Address]\n Москва, улица Обручева, 34/63\n ...",1.0
4,сеть быстрого питания,"[Address]\n Санкт-Петербург, 1-я Красноармейс...",1.0
...,...,...,...
495,наращивание ресниц,"[Address]\n Саратов, улица имени А.С. Пушкина...",1.0
496,игры,"[Address]\n Москва, Щёлковское шоссе, 79, кор...",0.0
497,домашний интернет в курске что подключить отзы...,"[Address]\n Курск, Садовая улица, 5\n [Name]...",0.0
498,гостиница волгодонск сауна номер телефона,"[Address]\n Ростовская область, городской окр...",0.0


In [9]:
# проверка правильности сборки обозначенного пайплайна
prev_res = ''
prev_queries = ''
queries_count = 0
prompt = f"""
Запрос пользователя: {data.iloc[498,0]}
Найденное место:
{data.iloc[498,1]}
"""
input_messages = [HumanMessage(content=prompt)]
output = graph.invoke({"messages": input_messages})
print(output)

{'messages': [HumanMessage(content='\nЗапрос пользователя: гостиница волгодонск сауна номер телефона\nНайденное место:\n[Address]\n  Ростовская область, городской округ Волгодонск, Лодочная улица, 15А\n  [Name]\n  Поплавок; Poplavok\n  [Normalized_name]\n  База , дом отдыха\n  [Description]\n  Предлагает размещение в различных типах жилья, доступ к беседке с видом на Дон и услугам банного комплекса | беседка с видом на Дон | дом «Шалаш» | номер на двоих | VIP дом | вход в банный комплекс\n  [Reviews]\n  Организация «Поплавок» предлагает услуги базы отдыха с банями, бассейнами и рестораном. Отзывы смешанные: много положительных оценок, но есть и существенные претензии к сервису и кухне. Хвалят: бани, бассейны, территорию, сервис. Критикуют: высокие цены, проблемы с обслуживанием в ресторане, ограничения на пляже, неудобство для семейного отдыха.\n\n| 1. Хвалят баню | 2. Общий положительный отзыв | 3. Положительный отзыв | 4. Отмечают приветливый персонал | 5. Критикуют отсутствие сушилк

In [29]:
# начало последовательной обработки тестовой выборки
data['advanced']=2.0
ok = 0
total = 0
for i in range(len(data)):
    total+=1
    prev_res = ''
    prev_queries = ''
    queries_count = 0
    prompt = f"""
    Запрос пользователя: {data.iloc[i,0]}
    Найденное место:
    {data.iloc[i,1]}
    """
    input_messages = [HumanMessage(content=prompt)]
    output = graph.invoke({"messages": input_messages})
    answer_number = extract_answer_number(output)
    print(queries_count, len(prev_res)) # отслеживаем, что модель действительно ходит в поиск
    data.iloc[i,-1] = answer_number
    if data.iloc[i,-1]==data.iloc[i,2]:
       ok+=1
    print(ok/total)

1 1406
1.0
1 1110
1.0
1 1086
1.0
1 1079
1.0
0 0
1.0
0 0
1.0
0 0
1.0
1 716
0.875
0 0
0.8888888888888888
0 0
0.8
1 726
0.7272727272727273
1 716
0.75
1 707
0.6923076923076923
1 1365
0.7142857142857143


RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.0-flash', 'location': 'global'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '28s'}]}}]

In [102]:
# иногда возникает ошибка превышения лимита запросов в промежуток времени (как выше), в следующей версии агента будут попытки ее исправить
# но пока будем просто запоминать номер примера, на котором возникла ошибка и число правильных ответов
# поэтому это код, актуальный для последнего случая продолжения после ошибки
from tqdm.auto import tqdm
total=481
ok=291
for i in tqdm(range(481,len(data))):
    total+=1
    prev_res = ''
    prev_queries = ''
    queries_count = 0
    prompt = f"""
    Запрос пользователя: {data.iloc[i,0]}
    Найденное место:
    {data.iloc[i,1]}
    """
    input_messages = [HumanMessage(content=prompt)]
    output = graph.invoke({"messages": input_messages})
    answer_number = extract_answer_number(output)
    data.iloc[i,-1] = answer_number
    if data.iloc[i,-1]==data.iloc[i,2]:
       ok+=1
    print(ok/total, queries_count)

  0%|          | 0/19 [00:00<?, ?it/s]

0.6058091286307054 1
0.6045548654244306 1
0.6053719008264463 1
0.6061855670103092 1
0.6069958847736625 2
0.6078028747433265 1
0.6065573770491803 1
0.6053169734151329 1
0.6061224489795919 2
0.6069246435845214 1
0.6077235772357723 1
0.6064908722109533 1
0.6072874493927125 2
0.6080808080808081 2
0.6068548387096774 1
0.6056338028169014 1
0.6044176706827309 2
0.6032064128256514 1
0.604 2


In [103]:
count=0
ok=0
for i in range(len(data)):
    if data.iloc[i,-1]!=2:
        count+=1
        if data.iloc[i,-1]==data.iloc[i,2]:
            ok+=1
    else:
        break
print(count, ok)

500 302


In [11]:
acc = 302/500
print(acc)

0.604


In [13]:
# в оригинальном коде с экспериментами сначала проводился опыт с теми же моделями, но вместо Tavily использовался DuckDuckGo и в поисковик запрос вводился автоматически скриптом (весь код этого нулевого эксперимента в отдельном ноутбуке). Еще промпт был проще
data.to_csv('eval_gemini_exp1_exp2.csv',index=False)
data = pd.read_csv("/kaggle/input/dls-project-ds/eval_gemini_exp1_exp2.csv")
# поэтому результаты нулевого эксперимента (gemini_2.5_flash_exp1), первого эксперимента (gemini_2.0_flash_exp2) и бейзлайна сохранены следующим образом:
data

,request,response,target,gemini_2.5_flash_exp1,gemini_2.0_flash_exp2,baseline
0,сигары,"[Address]\n Москва, Дубравная улица, 34/29\n ...",1.0,1.0,1.0,1.0
1,кальянная спб мероприятия,"[Address]\n Санкт-Петербург, Большой проспект...",0.0,0.0,0.0,0.0
2,Эпиляция,"[Address]\n Московская область, Одинцово, ули...",1.0,1.0,1.0,1.0
3,стиральных машин,"[Address]\n Москва, улица Обручева, 34/63\n ...",1.0,1.0,1.0,1.0
4,сеть быстрого питания,"[Address]\n Санкт-Петербург, 1-я Красноармейс...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
495,наращивание ресниц,"[Address]\n Саратов, улица имени А.С. Пушкина...",1.0,1.0,0.0,0.0
496,игры,"[Address]\n Москва, Щёлковское шоссе, 79, кор...",0.0,1.0,1.0,1.0
497,домашний интернет в курске что подключить отзы...,"[Address]\n Курск, Садовая улица, 5\n [Name]...",0.0,0.0,1.0,0.0
498,гостиница волгодонск сауна номер телефона,"[Address]\n Ростовская область, городской окр...",0.0,1.0,1.0,1.0


In [14]:
ok_b = 0
ok_exp0 = 0
ok_exp1 = 0
total = 0
for i in range(len(data)):
    total+=1
    if data.iloc[i, 3]==data.iloc[i, 2]:
        ok_exp0+=1
    if data.iloc[i, 4]==data.iloc[i, 2]:
        ok_exp1+=1
    if data.iloc[i, 5]==data.iloc[i, 2]:
        ok_b+=1
print(ok_b/total, ok_exp0/total, ok_exp1/total)

0.758 0.648 0.604


## Проведем переоценку по обновленным целевым меткам

In [3]:
import pandas as pd
data = pd.read_csv("/kaggle/input/eval_gemini_exp1_exp2.csv")

In [4]:
ok_b = 0
ok_exp0 = 0
ok_exp1 = 0
total = 0
for i in range(len(data)):
    total+=1
    if data.iloc[i, 3]==data.iloc[i, 2]:
        ok_exp0+=1
    if data.iloc[i, 4]==data.iloc[i, 2]:
        ok_exp1+=1
    if data.iloc[i, 5]==data.iloc[i, 2]:
        ok_b+=1
print(ok_b/total, ok_exp0/total, ok_exp1/total)

0.71 0.764 0.684


## Accuracy первой версии полноценного агента: 0.684 (хуже бейзлайна)

## Нулевой эксперимент пока лучший из агентоориентированных (из-за простого промпта без "переобучения" на некоторых примерах из трейна и использования другого поисковика, который предоставляет субъективно более чистые и конкретные данные)

# Задача следующей версии агента - достичь или побить результат нулевого эксперимента (0.764)